In [9]:
from skimage import io
import random
import os
import albumentations as A
import cv2
import matplotlib.pyplot as plt

##### Usefull paths

In [14]:
imgs_path = os.path.join('data', 'originals','imgs')
masks_path = os.path.join('data', 'originals', 'masks')
imgs_resized_path = os.path.join('data', 'originals', 'imgs_resized')
masks_resized_path = os.path.join('data', 'originals', 'masks_resized')
augmented_imgs_path = os.path.join('data','augmented', 'aug_imgs')
augmented_masks_path = os.path.join('data', 'augmented', 'aug_masks')

In [8]:
for directory in [augmented_imgs_path, augmented_masks_path, imgs_path, masks_path]:
    for f in os.listdir(directory):
        full_path = os.path.join(directory, f)
        os.remove(full_path)

##### Resize images

In [15]:
# Tamaño deseado para las imágenes
desired_width = 256
desired_height = 256

# Iterar sobre todas las imágenes en el directorio de entrada
for img in os.listdir(imgs_path):
    if img.endswith('.png') or img.endswith('.jpeg'):
        # Leer la imagen
        image = cv2.imread(os.path.join(imgs_path, img))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resized_img = image[10:266, 0:256]
        cv2.imwrite(os.path.join(imgs_resized_path, img), resized_img)

# Iterar sobre todas las máscaras en el directorio de entrada
for mask in os.listdir(masks_path):
    if mask.endswith('.png') or mask.endswith('.jpeg'):
        # Leer la máscara
        mask_img = cv2.imread(os.path.join(masks_path, mask))
        image_rgb = cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB)
        resized_mask = cv2.resize(mask_img, (desired_width, desired_height))
        cv2.imwrite(os.path.join(masks_resized_path, mask), resized_mask)
    else:
        print(mask)
    

In [18]:
file_index = 0
for sat,num in [('L5',20), ('L8',48),('L9',6)]:
    for i in range(num):
        try:
            img = os.path.join(imgs_path, f'{sat}_{i}_RGB.png')
            mask = os.path.join(masks_path, f'{sat}_{i}_mask.png')

            image = cv2.imread(img)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized_img = image[10:266, 0:256]
            cv2.imwrite(os.path.join(imgs_resized_path, f'{file_index}.png'), resized_img)
            print(image)
            mask_img = cv2.imread(mask)
            image_rgb = cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB)
            resized_mask = cv2.resize(mask_img, (desired_width, desired_height))
            cv2.imwrite(os.path.join(masks_resized_path, f'{file_index}.png'), resized_mask)
            print(mask_img)
            file_index += 1
        except FileNotFoundError:
            continue
        

[ WARN:0@3216.586] global loadsave.cpp:248 findDecoder imread_('data/originals/imgs/L5_0_RGB.png'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


##### Create and save augmented images

In [ ]:
images_to_generate = 75

aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2)
])

images = []
masks = []


i = 1
file_index = 301
num_images = len(os.listdir(imgs_path))
while i <= images_to_generate:
    number = random.randint(0, num_images - 1)
    try:
        image = os.path.join(imgs_path, f'{number}_L5_RGB.png')
        mask = os.path.join(masks_path, f'{number}_L5_mask.png')
        original_image = io.imread(image)
        original_mask = io.imread(mask)
        
        augmented = aug(image=original_image, mask=original_mask)
        transformed_image = augmented['image']
        transformed_mask = augmented['mask']

        new_image_path= "%s/%s.png" %(augmented_imgs_path, file_index)
        new_mask_path = "%s/%s.png" %(augmented_masks_path, file_index)
        io.imsave(new_image_path, transformed_image)
        io.imsave(new_mask_path, transformed_mask)
    except FileNotFoundError:
        print(f'File not found: {number}')
        continue
    i =i+1
    file_index += 1

**This code has been obtained from [this video](https://www.youtube.com/watch?v=J_XSd_u_Yew&t=359s)